In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rahmatsyahfirdaus","key":"d538092771bd326e064f3bd506fe9fd3"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                               title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
Cornell-University/arxiv                                          arXiv Dataset                                     877MB  2020-08-14 23:50:57           1727  
agirlcoding/all-space-missions-from-1957                          All Space Missions from 1957                      101KB  2020-08-13 16:18:58            675  
landlord/handwriting-recognition                                  Handwriting Recognition                             1GB  2020-08-05 17:20:36            235  
jmmvutu/summer-products-and-sales-in-ecommerce-wish               Sales of summer clothes in E-commerce Wish        351KB  2020-08-06 22:27:25            631  
andrewmvd/heart-failure-clinical-data   

In [ ]:
!kaggle datasets download -d zarajamshaid/language-identification-datasst

 90% 5.00M/5.53M [00:00<00:00, 14.1MB/s]
100% 5.53M/5.53M [00:00<00:00, 14.0MB/s]


In [ ]:
# melakukan ekstraksi pada file zip
import zipfile,os
file_zip = '/content/language-identification-datasst.zip'
zip_ref = zipfile.ZipFile(file_zip, 'r')
zip_ref.extractall('/content/dataset_bahasa')
zip_ref.close()

In [ ]:
import pandas as pd
df = pd.read_csv('dataset_bahasa/dataset.csv')

In [ ]:
df.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch


In [ ]:
language = pd.get_dummies(df.language)
df_baru = pd.concat([df, language], axis=1)
df_baru = df_baru.drop(columns='language')
df_baru

,Text,Arabic,Chinese,Dutch,English,Estonian,French,Hindi,Indonesian,Japanese,Korean,Latin,Persian,Portugese,Pushto,Romanian,Russian,Spanish,Swedish,Tamil,Thai,Turkish,Urdu
0,klement gottwaldi surnukeha palsameeriti ning ...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,sebes joseph pereira thomas på eng the jesuit...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,de spons behoort tot het geslacht haliclona en...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,hors du terrain les années et sont des année...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21996,ใน พศ หลักจากที่เสด็จประพาสแหลมมลายู ชวา อินเ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
21997,con motivo de la celebración del septuagésimoq...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
21998,年月，當時還只有歲的她在美國出道，以mai-k名義推出首張英文《baby i like》，由...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
text = df_baru['Text'].values
#language = df_baru[['Arabic',	'Chinese'	,'Dutch'	,'English'	,'Estonian'	,'French'	,'Hindi'	,'Indonesian'	,'Japanese'	,'Korean'	,'Latin'	,'Persian'	,'Portugese'	,'Pushto'	,'Romanian'	,'Russian'	,'Spanish'	,'Swedish'	,'Tamil'	,'Thai'	,'Turkish'	,'Urdu']].values
language = df_baru.values[:, 1:].astype(float)

In [ ]:
text

array(['klement gottwaldi surnukeha palsameeriti ning paigutati mausoleumi surnukeha oli aga liiga hilja ja oskamatult palsameeritud ning hakkas ilmutama lagunemise tundemärke  aastal viidi ta surnukeha mausoleumist ära ja kremeeriti zlíni linn kandis aastatel – nime gottwaldov ukrainas harkivi oblastis kandis zmiivi linn aastatel – nime gotvald',
       'sebes joseph pereira thomas  på eng the jesuits and the sino-russian treaty of nerchinsk  the diary of thomas pereira bibliotheca instituti historici s i --   rome libris ',
       'ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เริ่มตั้งแต่ถนนสนามไชยถึงแม่น้ำเจ้าพระยาที่ถนนตก กรุงเทพมหานคร เป็นถนนรุ่นแรกที่ใช้เทคนิคการสร้างแบบตะวันตก ปัจจุบันผ่านพื้นที่เขตพระนคร เขตป้อมปราบศัตรูพ่าย เขตสัมพันธวงศ์ เขตบางรัก เขตสาทร และเขตบางคอแหลม',
       ...,
       'con motivo de la celebración del septuagésimoquinto ° aniversario de la fundación del departamento en  guillermo ceballos espinosa presentó a la gobernación de caldas por encargo de su t

In [ ]:
from sklearn.model_selection import train_test_split

text_latih, text_test, language_latih, language_test = train_test_split(text, language, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='oov')
tokenizer.fit_on_texts(text_latih) 
# fit_on_text tidak dilakukan pada data test untuk mengetahui apakah model dapat melakukan prediksi dengan baik walaupun wordnya tidak diketahui oleh model
 
sekuens_latih = tokenizer.texts_to_sequences(text_latih)
sekuens_test = tokenizer.texts_to_sequences(text_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(22, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9 and logs.get('val_accuracy')>0.9):
      print("\nAkurasi telah mencapai >90%!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
history = model.fit(padded_latih, language_latih, epochs=30, 
                    validation_data=(padded_test, language_test), verbose=2, callbacks=[callbacks])

Epoch 1/30
550/550 - 9s - loss: 2.2703 - accuracy: 0.2087 - val_loss: 1.2827 - val_accuracy: 0.5555
Epoch 2/30
550/550 - 8s - loss: 1.2824 - accuracy: 0.4896 - val_loss: 0.7767 - val_accuracy: 0.6848
Epoch 3/30
550/550 - 8s - loss: 0.8023 - accuracy: 0.6859 - val_loss: 0.4217 - val_accuracy: 0.8616
Epoch 4/30
550/550 - 8s - loss: 0.5434 - accuracy: 0.8114 - val_loss: 0.3964 - val_accuracy: 0.8652
Epoch 5/30
550/550 - 8s - loss: 0.4189 - accuracy: 0.8621 - val_loss: 0.2526 - val_accuracy: 0.9166
Epoch 6/30
550/550 - 8s - loss: 0.3593 - accuracy: 0.8824 - val_loss: 0.2584 - val_accuracy: 0.9177
Epoch 7/30
550/550 - 8s - loss: 0.3136 - accuracy: 0.8932 - val_loss: 0.3056 - val_accuracy: 0.9068
Epoch 8/30

Akurasi telah mencapai >90%!
550/550 - 8s - loss: 0.2846 - accuracy: 0.9019 - val_loss: 0.2360 - val_accuracy: 0.9259


In [ ]:
import numpy as np
prediksi = ["data on kaggle can be used by scientist all over the world"]
sekuence_prediksi = tokenizer.texts_to_sequences(prediksi)# kita tidak menggunakan fit_on_text pada data test untuk mengetahui apakah model dapat melakukan prediksi dengan baik walaupun wordnya tidak diketahui oleh model
padded_presiksi = pad_sequences(sekuence_prediksi)

hasilPrediksi = model.predict(padded_presiksi)

output = np.max(hasilPrediksi)
output = np.where(hasilPrediksi == output)
index = output[1][0]

bahasa = ['Arabic',	'Chinese'	,'Dutch'	,'English'	,'Estonian'	,'French'	,'Hindi'	,'Indonesian'	,'Japanese'	,'Korean'	,'Latin'	,'Persian'	,'Portugese'	,'Pushto'	,'Romanian'	,'Russian'	,'Spanish'	,'Swedish'	,'Tamil'	,'Thai'	,'Turkish'	,'Urdu']
print('{} = {}'.format(prediksi[0], bahasa[index]))
print('Language       Probability\n----------------------------\n{}     {:.2f}'.format(bahasa[index],hasilPrediksi[0][index]))

data on kaggle can be used by scientist all over the world = English
Language       Probability
----------------------------
English     0.99
